In [ ]:
for text in llm("AI is going to", stream=True):
    print(text, end="", flush=True)


# results on the cpu using ctransformers
### - 27.4 seconds
### output:
<p>
have an enormous impact on the future of the industry.
This year, AI was all over the show floor and there are a number of companies that are using AI in their products. Here's a quick look at some of them:
Fulcrum is an automated, artificial intelligence-powered software solution that helps manufacturers improve productivity and efficiency by automating processes, removing errors, and freeing up employees to focus on more complex tasks.
Kapil Khera, co-founder and CEO of Fulcrum, told Inman: "I don't know a single company that doesn't have AI in their product. I think it's the most important thing to come out of 2018."
The company is currently working on machine learning tools and workflow applications with machine intelligence technology so you can really begin building these powerful AI-powered solutions. The full AI-focused ecosystem, with automation in product design and sophisticated analytics, will be available to all customers by the end of 2018.
Kapil Khera, co-founder and CEO of Fulcrum
Surely
</p>

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name_or_path = "TheBloke/TinyLlama-1.1B-Chat-v1.0-GPTQ"
# To use a different branch, change revision
# For example: revision="gptq-4bit-32g-actorder_True"
model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)


In [ ]:

def get_response_tiny_llama(text):
    path='/teamspace/studios/this_studio/TinyLlama-1.1B-Chat-v1.0-GPTQ'
    model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

    tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)
    prompt = text
    system_message = "You are a story writing assistant"
    prompt_template=f'''<|system|>
    {system_message}</s>
    <|user|>
    {prompt}</s>
    <|assistant|>
    '''

    print("\n\n*** Generate:")

    input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
    output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
    return(tokenizer.decode(output[0]))

In [ ]:
print(get_response_tiny_llama("what is qlora?"))

# result on gpu 2.8 seconds

# handling the embedding model

In [ ]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted"]

model = SentenceTransformer('sentence-transformers/all-MiniLM-L12-v2')
embeddings = model.encode(sentences)
print(embeddings)


In [ ]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')

import os
pdf_list=os.listdir("/Users/matansharon/python/opensource_rag/pdf_files")
text_list=os.listdir("/Users/matansharon/python/opensource_rag/text_files")

for pdf in pdf_list:
    temp=pdf.replace(".pdf",".txt")
    if temp not in text_list:
        text=convert_from_pdf_to_text("pdf_files/"+pdf)
        with open("text_files/"+temp, 'w') as f:
            f.write(text)

In [ ]:
def convert_from_pdf_to_text(file_path):
    import PyPDF2

    read_pdf = PyPDF2.PdfReader(file_path)
    pages = read_pdf.pages
    res=''
    for page in pages:
        res+=page.extract_text()
    return res
pages=convert_from_pdf_to_text('/Users/matansharon/python/opensource_rag/pdf_files/A semantic loss for ontology classification.pdf')


In [ ]:
import chromadb
client=chromadb.Client()
import os
import chromadb.utils.embedding_functions as embedding_functions
huggingface_ef = embedding_functions.HuggingFaceEmbeddingFunction(
    api_key=os.environ.get("HuggingFace_API_KEY"),
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

collection=client.get_or_create_collection('test',embedding_function=huggingface_ef)


In [7]:
import chromadb
persist_directory = "chroma_storage"
client = chromadb.PersistentClient(path=persist_directory)
collection=client.get_collection("file1_collection")
print(collection.get().keys())



dict_keys(['ids', 'embeddings', 'metadatas', 'documents', 'uris', 'data'])


In [44]:
results

{'ids': [['45', '46', '3', '25', '22']],
 'distances': [[0.62945196401265,
   1.2014087659918908,
   1.4490561050844064,
   1.457502030657284,
   1.4734288925551329]],
 'metadatas': [[{'filename': 'Component Separation method for CMB using Convolutional Neural Networks.pdf',
    'page_number': 0},
   {'filename': 'Component Separation method for CMB using Convolutional Neural Networks.pdf',
    'page_number': 1},
   {'filename': 'qlora.pdf', 'page_number': 3},
   {'filename': 'qlora.pdf', 'page_number': 25},
   {'filename': 'qlora.pdf', 'page_number': 22}]],
 'embeddings': None,
 'documents': [['Component Separation method for CMB using Convolutional Neural Networks\nA. Quintana1,2,3, B. Ruiz-Granados4, P. Ruiz-Lapuente1,2\n1Instituto de F´ ısica Fundamental (IFF-CSIC), Madrid, Spain,\n2Institut de Ci` encies del Cosmos (UB-IEEC), Barcelona, Spain\n3Universidad Internacional de Valencia, Valencia, Spain\n4Universidad de C´ ordoba, C´ ordoba, Spain\nThe aim of this project is to recover

In [43]:
results = collection.query(
            query_texts=["summerize the Component Separation method for CMB using Convolutional Neural Networks document"], n_results=5, include=["documents", "metadatas","distances"])
print(results)
for i in range(len(results['distances'][0])):
    if (abs(results['distances'][0][i]-1)<0.4):
        print(results['distances'][0][i])
        print(results['documents'][0][i][:100])
        print(results['metadatas'][0][i])
        print("-"*100)

{'ids': [['45', '46', '3', '25', '22']], 'distances': [[0.62945196401265, 1.2014087659918908, 1.4490561050844064, 1.457502030657284, 1.4734288925551329]], 'metadatas': [[{'filename': 'Component Separation method for CMB using Convolutional Neural Networks.pdf', 'page_number': 0}, {'filename': 'Component Separation method for CMB using Convolutional Neural Networks.pdf', 'page_number': 1}, {'filename': 'qlora.pdf', 'page_number': 3}, {'filename': 'qlora.pdf', 'page_number': 25}, {'filename': 'qlora.pdf', 'page_number': 22}]], 'embeddings': None, 'documents': [['Component Separation method for CMB using Convolutional Neural Networks\nA. Quintana1,2,3, B. Ruiz-Granados4, P. Ruiz-Lapuente1,2\n1Instituto de F´ ısica Fundamental (IFF-CSIC), Madrid, Spain,\n2Institut de Ci` encies del Cosmos (UB-IEEC), Barcelona, Spain\n3Universidad Internacional de Valencia, Valencia, Spain\n4Universidad de C´ ordoba, C´ ordoba, Spain\nThe aim of this project is to recover the CMB anisotropies maps in temper

In [ ]:
#this cell is a testing cell for filtering the results of the query by the distance

temp_results={"ids":[[]],"distances":[[]],"metadatas":[[]],"documents":[[]]}
        for i in range(5):
            if abs(1-results["distances"][0][i]<0.4): 
                temp_results["ids"][0].append(results["ids"][0][i])
                temp_results["distances"][0].append(results["distances"][0][i])
                temp_results["metadatas"][0].append(results["metadatas"][0][i])
                temp_results["documents"].append(results["documents"][0][i])
        results=temp_results

In [ ]:
all_file_names=set()
for line in collection.get()['metadatas']:
    all_file_names.add(line['filename'])
all_file_names

In [ ]:
for doc in collection.get()['documents']:
    print(len(doc))

In [3]:
import PyPDF2
pages=PyPDF2.PdfReader("/Users/matansharon/python/opensource_rag/pdf_files/qlora.pdf").pages


In [6]:
from nltk.tokenize import word_tokenize
for page in pages:
    print(len(word_tokenize(page.extract_text())))

510
854
478
777
720
739
668
725
968
821
617
570
528
788
742
616
787
759
772
832
89
572
626
566
265
254
